## Describe the current simulation

In [1]:
simulation_description = "Test V1V2H1 - different initial position - rn + diffusion ON. No attraction to V2. Run 5fois plus longue"

## Import initial parameters

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from spatial_parms import initialize_spatial_parms
from parameters_animals import initialize_animal_parms
from parameters_vegetation import initialize_veg_parms
from initial_distribution import initial_sp_distribution
from Diffusion_parameters import initialize_diffusion_parms
from intermediate import log_intermediates

Nx, Ny, dx, dy, dt, Nt, Lx, Ly = initialize_spatial_parms()
V1, V2, H1, H2, P, k_V1, k_V2, mask_V2, mask_V1 = initial_sp_distribution(Nx, Ny)
V2_croiss, V1_croiss, k_V2_val, k_V1_norm, k_V2_max = initialize_veg_parms()
rho_H1, a_H2, a_H1, h_V2H2, e_V1, e_V2, h_V1H1, h_V2H1, h_V1H2, mu_H2, mu_H1, epsi_AJ, chi_H2, chi_H1, a_PH2, a_PH1, h_PH1, h_PH2, mu_P, phi_P, h_P, chi_P, epsi_H1H2, gamma_H1H2, I_H1, I_H2 = initialize_animal_parms(k_V2_max, k_V1_norm, k_V2_max)
sigma_H1, sigma_H2, sigma_P, eta_H1, eta_H2, eta_P, alpha_H2H2, alpha_H1H1, alpha_PP, alpha_H2V1, alpha_H2V2, alpha_H1V2, alpha_H1V1, alpha_PH2, alpha_PH1 = initialize_diffusion_parms()


## Save metadata associated with the simulation

In [3]:
from log_metadata import save_metadata
from datetime import datetime

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")

# Duration of the simulations
t_span = (0, dt * Nt)

# Present species
species = {"V1": V1, "V2": V2, "H1": H1, "H2": H2, "P": P}
active_species = [name for name, field in species.items() if np.any(field != 0)]

# Parameters we want to track
params = {
    "alpha_H1V1": alpha_H1V1,
    "alpha_H1V2": alpha_H1V2,
    "alpha_H2V2": alpha_H2V2,
    "alpha_H2V1": alpha_H1V1,

    "alpha_H2H2": alpha_H2H2,
    "alpha_H1H1": alpha_H1H1,
    
    "eta_H1": eta_H1,
    "eta_H2": eta_H1,
    "eta_P": eta_P,

    "sigma_H1": sigma_H1,
    "sigma_H2": sigma_H2,
    "sigma_P": sigma_P,

    "alpha_PH2": alpha_PH2,
    "alpha_PH1": alpha_PH1
}


grid = {"Nx": Nx, "Ny": Ny, "dx": dx, "dy": dy}
sim_settings = {"Nt": Nt, "t_span": list(t_span), "solver": "LSODA"}
notes = simulation_description

metadata_path = save_metadata(active_species, params, grid, sim_settings, timestamp, notes)


## Define landscape barrier

In [4]:
from Landscape_disturbances_mask import create_barrier_mask
import matplotlib.pyplot as plt

barrier_mask = create_barrier_mask((Nx, Ny), orientation='vertical', thickness=0, position='center')
# Put thickness to 0 to get ride of the barrier


## Plot the initial distribution of species, carrying capacities and barrier

In [5]:
from plot_distribution import plot_species_distribution
import os
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# Base folder for all time series plots
plot_dir = os.path.join("outputs", timestamp, "initial_distribution")
os.makedirs(plot_dir, exist_ok=True) # Make sure the folder exist, otherwise, gets created

t_idx = 0

V1_t = V1[:, :]
V2_t = V2[:, :]
H1_t = H1[:, :]
H2_t = H2[:, :]
P_t  = P[:, :]
barrier_mask  = barrier_mask[:, :]
k_V2  = k_V2[:, :]
k_V1  = k_V1[:, :]

species_fields = { 
    "Deciduous": V2_t,
    "Lichen": V1_t,
    "Caribou": H1_t,
    "Moose": H2_t,
    "Predator": P_t,
    "Barrier": barrier_mask,
    "k_V2": k_V2,
    "k_V1": k_V1
}


for species_name, field in species_fields.items():
    if np.any(field != 0):
        filename = os.path.join(plot_dir, f"{species_name}.png")
        plot_species_distribution(field, Lx, Ly, species_name=species_name, save_path=filename, time_label= t_idx)



## Plot the patches scores for herbivores

In [6]:
import matplotlib.pyplot as plt
from compute_score_maps import compute_norm_score_patch

# Base folder for all time series plots
score_dir = os.path.join("outputs", timestamp, "score_maps")
os.makedirs(score_dir, exist_ok=True) # Make sure the folder exist, otherwise, gets created

# Compute score for the herbivore H2
score_G_H2, norm_score_G_H2 = compute_norm_score_patch(
    alpha_H2H2, H2,
    alpha_H2V1, V1,
    alpha_H2V2, V2,
    alpha_PH2, P
)

# Compute score for the herbivore H1
score_G_H1, norm_score_G_H1 = compute_norm_score_patch(
    alpha_H1H1, H1,
    alpha_H1V1, V1,
    alpha_H1V2, V2,
    alpha_PH2, P
)

# Compute score for the predator P
score_G_P, norm_score_G_P = compute_norm_score_patch(
    alpha_PP, P,
    alpha_PH1, H1,
    alpha_PH2, H2,
    0, 0
)

fields_to_plot = {
    "V1_distribution": V1,
    "V2_distribution": V2,
    "H1_distribution": H1,
    "H2_distribution": H2,
    "G_H2_raw": score_G_H2,
    "G_H1_raw": score_G_H1,
    "G_P_raw": score_G_P,
    "G_H2_normalized": norm_score_G_H2,
    "G_H1_normalized": norm_score_G_H1,
    "G_P_normalized": norm_score_G_P
}

colormaps = {
    "V1_distribution": "viridis",
    "V2_distribution": "viridis",
    "H1_distribution": "viridis",
    "H2_distribution": "viridis",
    "G_H2_raw": "viridis",
    "G_H1_raw": "viridis",
    "G_P_raw": "viridis",
    "G_H2_normalized": "plasma",
    "G_H1_normalized": "plasma",
    "G_P_normalized": "plasma"
    
}


for name, field in fields_to_plot.items():
    cmap = colormaps.get(name, "viridis")  # fallback to viridis
    plt.figure()
    plt.imshow(field, origin="lower", cmap=cmap)
    plt.colorbar(label="Score")
    plt.title(name.replace("_", " "))
    plt.savefig(os.path.join(score_dir, f"{name}.png"), bbox_inches="tight", dpi=300)
    plt.close()



## Solve the PDE

In [7]:
from scipy.integrate import solve_ivp
from PDE_solving import system_rhs
import numpy as np
from fct_flat_field import flatten_fields
from fct_flat_field import unflatten_fields
from intermediate import log_intermediates


# Prep initial state
y0 = flatten_fields(V1, V2, H1, H2, P)

print("y0 has NaNs?", np.any(np.isnan(y0)))
print("y0 has Infs?", np.any(np.isinf(y0)))
print("y0 min/max:", np.min(y0), np.max(y0))

t_span = (0, dt * Nt)
t_eval = np.linspace(*t_span, Nt)

# Bundle parameters
params = {
    'V1_croiss': V1_croiss, 'V2_croiss': V2_croiss,
    'k_V1_norm': k_V1_norm, 'k_V2_val': k_V2_val,
    'a_H1': a_H1, 'a_H2': a_H2,
    'h_V1H1': h_V1H1, 'h_V2H1': h_V2H1,
    'h_V1H2': h_V1H2, 'h_V2H2': h_V2H2,
    'rho_H1': rho_H1,
    'h_PH1': h_PH1, 'h_PH2': h_PH2,
    'a_PH1': a_PH1, 'a_PH2': a_PH2, 
    'chi_H1': chi_H1, 'chi_H2': chi_H2,
    'mu_H1': mu_H1, 'mu_H2': mu_H2, 
    'e_V1': e_V1, 'e_V2': e_V2,
    'epsi_AJ': epsi_AJ, 
    'sigma_H1': sigma_H1, 'sigma_H2': sigma_H2, 'sigma_P': sigma_P, 
    'eta_H1': eta_H1, 'eta_H2': eta_H2, 'eta_P': eta_P, 
    'alpha_H1H1': alpha_H1H1, 'alpha_H2H2': alpha_H2H2, 'alpha_PP': alpha_PP,
    'alpha_H2V1': alpha_H2V1, 'alpha_H2V2': alpha_H2V2,
    'alpha_H1V1': alpha_H1V1, 'alpha_H1V2': alpha_H1V2,
    'alpha_PH1': alpha_PH1, 'alpha_PH2': alpha_PH2,
    'mu_P': mu_P, 'phi_P': phi_P, 'h_P': h_P,
    'chi_P': chi_P, 'epsi_H1H2': epsi_H1H2,
    'gamma_H1H2': gamma_H1H2
    
}

# 4. Initialize logs BEFORE defining the function
logged_data = {
    "score_G_H1": [],
    "Dm_eff_H1": [],
    "score_G_H2": [],
    "Dm_eff_H2": [],
    "score_G_P": [],
    "Dm_eff_P": [],
    "k_H1": [],
    "k_H2": []
}

# 5. Define the logging function

# 6. Call solver and pass log function
sol = solve_ivp(
    fun=lambda t, y: system_rhs(
        t, y, Nx, Ny, dx, dy, params, mask_V2, mask_V1, barrier_mask,
        log_fn=lambda t, **fields: log_intermediates(t, logged_data, **fields)
    ),
    t_span=t_span,
    y0=y0,
    t_eval=t_eval,
    method='LSODA'
)


y0 has NaNs? False
y0 has Infs? False
y0 min/max: 0.0 100000.0


## Transform the outputs for visualisation

In [10]:
import numpy as np

# Number of spatial points
N = Nx * Ny

# Convert sol.y to numpy array (if not already)
y_array = np.array(sol.y)

# Reshape each species
V1_over_time = y_array[0        : N,     :].reshape(Nx, Ny, -1)  # shape (Nx, Ny, Nt)
V2_over_time = y_array[N        : 2*N,   :].reshape(Nx, Ny, -1)
H1_over_time = y_array[2*N      : 3*N,   :].reshape(Nx, Ny, -1)
H2_over_time = y_array[3*N      : 4*N,   :].reshape(Nx, Ny, -1)
P_over_time  = y_array[4*N      : 5*N,   :].reshape(Nx, Ny, -1)


## Plot the species dynamics through time

In [11]:
import os
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# Base folder for all time series plots
base_output_dir = os.path.join("outputs", f"{timestamp}", "global_time_series")
os.makedirs(base_output_dir, exist_ok=True)

# Loop over species
species_list = ["H1", "H2", "V1", "V2", "P"]
for species_name in species_list:
    
    # Retrieve time series data
    species_field = globals()[f"{species_name}_over_time"]
    species_sum = np.sum(species_field, axis=(0, 1))

    # Set parameter values
    if species_name == "H1":
        alpha_V1 = alpha_H1V1
        alpha_V2 = alpha_H1V2
        eta = eta_H1
        color = "orange"
    elif species_name == "H2":
        alpha_V1 = alpha_H2V1
        alpha_V2 = alpha_H2V2
        eta = eta_H2
        color = "purple"
    elif species_name == "V1":
        alpha_V1 = alpha_V2 = eta = 0.0
        color = "blue"
    elif species_name == "V2":
        alpha_V1 = alpha_V2 = eta = 0.0
        color = "green"
    elif species_name == "P":
        alpha_V1 = alpha_PH1
        alpha_V2 = alpha_PH2
        eta = eta_P
        color = "red"
    else:
        continue

    # Create and save figure
    filename = f"{species_name}_alphaV1_{alpha_V1:.2f}_alphaV2_{alpha_V2:.2f}_eta_{eta:.2f}.png"
    filepath = os.path.join(base_output_dir, filename)

    plt.figure()
    plt.plot(sol.t, species_sum, label=species_name, color=color)
    plt.xlabel("Time")
    plt.ylabel("Total Value (Sum over Grid)")
    plt.title(f"{species_name} Dynamics Over Time")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(filepath)
    plt.close()


## Plot of the final spatial state

In [12]:
from plot_distribution import plot_species_distribution
import os
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime


# Base folder for all time series plots
plot_dir = os.path.join("outputs", timestamp, "final_distribution")
os.makedirs(plot_dir, exist_ok=True) # Make sure the folder exist, otherwise, gets created

max_t_idx = V2_over_time.shape[2] - 1
t_idx = max_t_idx

V1_t = V1_over_time[:, :, t_idx]
V2_t = V2_over_time[:, :, t_idx]
H1_t = H1_over_time[:, :, t_idx]
H2_t = H2_over_time[:, :, t_idx]
P_t  = P_over_time[:, :, t_idx]


species_fields = {
    "Deciduous": V2_t,
    "Lichen": V1_t,
    "Caribou": H1_t,
    "Moose": H2_t,
    "Predator": P_t,
    "Barrier": barrier_mask,
    "k_V2": k_V2,
    "k_V1": k_V1
}


for species_name, field in species_fields.items():
    if np.any(field != 0):
        filename = os.path.join(plot_dir, f"{species_name}.png")
        plot_species_distribution(field, Lx, Ly, species_name=species_name, save_path=filename, time_label= t_idx)



## Dynamics through time

In [17]:
import os
import matplotlib.pyplot as plt

# Choose your species
species_name = "H1"
species_field = globals()[f"{species_name}_over_time"]

# Extract parameter values associated with the species
if species_name == "H1":
    alpha_V1 = alpha_H1V1
    alpha_V2 = alpha_H1V2
    eta = eta_H1
elif species_name == "H2":
    alpha_V1 = alpha_H2V1
    alpha_V2 = alpha_H2V2
    eta = eta_H2
elif species_name == "P":
    alpha_V1 = alpha_PH1
    alpha_V2 = alpha_PH2
    eta = eta_P
else:
    raise ValueError(f"Unknown species: {species_name}")

# Create folder name from parameters

from datetime import datetime

# Base folder for all time series plots
output_dir = os.path.join("outputs", f"{timestamp}", f"frames_{species_name}")
os.makedirs(output_dir, exist_ok=True)

# Frame-saving function
def save_frame(field, title, t_idx, t_eval, Lx, Ly, filename):
    plt.imshow(field[:, :, t_idx], cmap='viridis', extent=[0, Lx, 0, Ly], origin='lower')
    plt.colorbar()
    plt.title(f"{title} at time t={t_eval[t_idx]:.2f}")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

# Save sampled frames
total_frames = 50
step = max(1, species_field.shape[2] // total_frames)
selected_indices = list(range(0, species_field.shape[2], step))[:total_frames]

for i, t_idx in enumerate(selected_indices):
    fname = os.path.join(output_dir, f"{species_name}_{i:04d}.png")
    save_frame(species_field, species_name, t_idx, t_eval, Lx, Ly, fname)


In [19]:
import imageio.v2 as imageio
import os

# Match the species name and corresponding parameters
species_name = "H1"

if species_name == "H1":
    alpha_V1 = alpha_H1V1
    alpha_V2 = alpha_H1V2
    eta = eta_H1
elif species_name == "H2":
    alpha_V1 = alpha_H2V1
    alpha_V2 = alpha_H2V2
    eta = eta_H2
elif species_name == "P":
    alpha_V1 = alpha_PH1
    alpha_V2 = alpha_PH2
    eta = eta_P
else:
    raise ValueError(f"Unknown species: {species_name}")

# Use the same parameter-aware folder as during frame generation

from datetime import datetime
output_dir = os.path.join("outputs", f"{timestamp}", f"frames_{species_name}")

# Output folder for GIFs
gif_dir = os.path.join("outputs", f"{timestamp}", "GIF")
os.makedirs(gif_dir, exist_ok=True)

# Number of frames and file loading
n_frames = 50
images = []
for i in range(n_frames):
    fname = os.path.join(output_dir, f"{species_name}_{i:04d}.png")
    images.append(imageio.imread(fname))

# Save the animation
output_gif_path = os.path.join(gif_dir, f"{species_name}.gif")
imageio.mimsave(output_gif_path, images, duration=0.2)


## Display dynamics of all species currently in the system

In [15]:
def save_overlay_frame(H1_field, H2_field, P_field, t_idx, t_eval, Lx, Ly, filename, barrier_mask):
    # Create figure
    plt.figure()

    # Plot H1 and assign mappable
    h1_plot = plt.imshow(H1_field[:, :, t_idx], cmap='Blues', alpha=0.6,
                         extent=[0, Lx, 0, Ly], origin='lower')

    # Plot H2
    plt.imshow(H2_field[:, :, t_idx], cmap='Reds', alpha=0.6,
               extent=[0, Lx, 0, Ly], origin='lower')
    
    # Plot P
    plt.imshow(P_field[:, :, t_idx], cmap='viridis', alpha=0.6,
               extent=[0, Lx, 0, Ly], origin='lower')

    # Overlay barrier
    plt.contour(barrier_mask, levels=[0.5], colors='black', linewidths=2,
                extent=[0, Lx, 0, Ly])

    # Plot details
    plt.title(f"H1, H2 and P overlay at t={t_eval[t_idx]:.2f}")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.colorbar(h1_plot)  # Explicit mappable from H1
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

# Folder for overlays
overlay_dir = os.path.join("outputs", timestamp, "frames_overlay_H1_H2_P")
os.makedirs(overlay_dir, exist_ok=True)


for i, t_idx in enumerate(selected_indices):
    fname = os.path.join(overlay_dir, f"H1_H2_P_overlay_{i:04d}.png")
    save_overlay_frame(H1_over_time, H2_over_time, P_over_time, t_idx, t_eval, Lx, Ly, fname, barrier_mask)



In [16]:
import imageio.v2 as imageio
import os

# Frame folder (overlay version)
overlay_dir = os.path.join("outputs", timestamp, "frames_overlay_H1_H2_P")
gif_output_dir = os.path.join("outputs", timestamp, "GIF")
os.makedirs(gif_output_dir, exist_ok=True)

# Assemble GIF from saved overlays
n_frames = 50  # match total_frames from frame creation
images = []
for i in range(n_frames):
    fname = os.path.join(overlay_dir, f"H1_H2_P_overlay_{i:04d}.png")
    images.append(imageio.imread(fname))

# Save the animation
gif_path = os.path.join(gif_output_dir, "H1_H2_P_overlay_animation.gif")
imageio.mimsave(gif_path, images, duration=0.2)


## DEBUG

In [ ]:
# Get the final logged carrying capacities
_, k_H1_final = logged_data["k_H1"][-1]
_, k_H2_final = logged_data["k_H2"][-1]

_, safe_k_H1_final = logged_data["safe_k_H1"][-1]
_, safe_k_H2_final = logged_data["safe_k_H2"][-1]



In [ ]:
_, k_H1_initial = logged_data["k_H1"][0]
_, k_H2_initial = logged_data["k_H2"][0]


In [ ]:
import matplotlib.pyplot as plt

def plot_carrying_capacities(k_H1, k_H2, Lx, Ly, time_label="final"):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    im1 = axes[0].imshow(k_H1, extent=[0, Lx, 0, Ly], origin="lower", cmap="Blues")
    axes[0].set_title(f"k_H1 (H1 Carrying Capacity) at t = {time_label}")
    axes[0].set_xlabel("x")
    axes[0].set_ylabel("y")
    plt.colorbar(im1, ax=axes[0])

    im2 = axes[1].imshow(k_H2, extent=[0, Lx, 0, Ly], origin="lower", cmap="Purples")
    axes[1].set_title(f"k_H2 (H2 Carrying Capacity) at t = {time_label}")
    axes[1].set_xlabel("x")
    axes[1].set_ylabel("y")
    plt.colorbar(im2, ax=axes[1])

    plt.tight_layout()
    plt.show()


In [ ]:
plot_carrying_capacities(k_H1_final, k_H2_final, Lx, Ly, time_label="final")
plot_carrying_capacities(safe_k_H1_final, safe_k_H2_final, Lx, Ly, time_label="final")


In [ ]:
plot_carrying_capacities(k_H1_initial, k_H2_initial, Lx, Ly, time_label="initial")


In [ ]:
print("k_H1_initial min:", np.min(k_H1_initial))
print("k_H1_initial max:", np.max(k_H1_initial))
